# Load Data từ Silver Layer sang Gold Layer (Dimension & Fact Tables)

Notebook này sẽ load dữ liệu từ Silver layer và transform sang Gold layer với:
- Dimension tables có surrogate key tự động tăng
- Fact tables với các foreign keys tương ứng

## 1. Import Libraries và Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session
spark = (
    SparkSession.builder
    .appName("Load_Silver_To_Gold")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    # Propagate environment variables to executors
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    # ===== Sử dụng JAR files local =====
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print(" Spark Session đã được khởi tạo!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

25/12/02 13:51:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


 Spark Session đã được khởi tạo!
Spark Master: spark://spark-master:7077
Application ID: app-20251202135125-0003


## 2. Load Dimension Tables từ Silver Layer

### 2.1. Dim_Time - Tạo bảng thời gian từ dữ liệu benchmark

In [2]:
print("Đang load Dim_Time...")

# Đọc dữ liệu từ Silver
df_benchmark = spark.table("nessie.silver_tables.benchmark")

# Tạo dim_time từ các year unique trong benchmark
df_time = df_benchmark.select("year").distinct()

# Tạo day, month từ year (giả sử ngày 1/1 của mỗi năm)
df_time = df_time.withColumn("day", lit(1)) \
    .withColumn("month", lit(1))

# Tạo timeKey tự động tăng
window_spec = Window.orderBy("year")
df_time = df_time.withColumn("timeKey", row_number().over(window_spec))

# Sắp xếp lại cột theo thứ tự
df_dim_time = df_time.select("timeKey", "day", "month", "year")

# Ghi vào Gold layer
df_dim_time.writeTo("nessie.gold_tables.dim_time") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_time.count()} dòng vào dim_time")
df_dim_time.show(10)

Đang load Dim_Time...


Đã load 5 dòng vào dim_time
+-------+---+-----+----+
|timeKey|day|month|year|
+-------+---+-----+----+
|      1|  1|    1|2021|
|      2|  1|    1|2022|
|      3|  1|    1|2023|
|      4|  1|    1|2024|
|      5|  1|    1|2025|
+-------+---+-----+----+



### 2.2. Dim_Region - Bảng khu vực

In [3]:
print("Đang load Dim_Region...")

# Đọc dữ liệu từ Silver
df_region_silver = spark.table("nessie.silver_tables.region")

# Tạo regionKey tự động tăng
window_spec = Window.orderBy("regionId")
df_dim_region = df_region_silver.withColumn("regionKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_region = df_dim_region.select("regionKey", "regionId", "regionName")

# Ghi vào Gold layer
df_dim_region.writeTo("nessie.gold_tables.dim_region") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_region.count()} dòng vào dim_region")
df_dim_region.show(10)

Đang load Dim_Region...


Đã load 64 dòng vào dim_region
+---------+--------+--------------------+
|regionKey|regionId|          regionName|
+---------+--------+--------------------+
|        1|      01|      Sở GDĐT Hà Nội|
|        2|      02|Sở GDĐT TP. Hồ Ch...|
|        3|      03|   Sở GDĐT Hải Phòng|
|        4|      04|     Sở GDĐT Đà Nẵng|
|        5|      05|    Sở GDĐT Hà Giang|
|        6|      06|    Sở GDĐT Cao Bằng|
|        7|      07|    Sở GDĐT Lai Châu|
|        8|      08|     Sở GDĐT Lào Cai|
|        9|      09| Sở GDĐT Tuyên Quang|
|       10|      10|    Sở GDĐT Lạng Sơn|
+---------+--------+--------------------+
only showing top 10 rows



### 2.3. Dim_School - Bảng trường đại học

In [4]:
print("Đang load Dim_School...")

# Đọc dữ liệu từ Silver
df_school_silver = spark.table("nessie.silver_tables.school")

# Tạo schoolKey tự động tăng
window_spec = Window.orderBy("schoolId")
df_dim_school = df_school_silver.withColumn("schoolKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_school = df_dim_school.select("schoolKey", "schoolId", "schoolName", "province")

# Ghi vào Gold layer
df_dim_school.writeTo("nessie.gold_tables.dim_school") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_school.count()} dòng vào dim_school")
df_dim_school.show(10)

Đang load Dim_School...
Đã load 265 dòng vào dim_school
+---------+--------+--------------------+--------+
|schoolKey|schoolId|          schoolName|province|
+---------+--------+--------------------+--------+
|        1|     ANH|Học viện An Ninh ...|  Hà Nội|
|        2|     ANS|Đại học An Ninh N...|  Hà Nội|
|        3|     BKA|Đại học Bách khoa...|  Hà Nội|
|        4|     BMU|Đại học Buôn Ma T...| Đắk Lắk|
|        5|     BPH| Học viện Biên Phòng|  Hà Nội|
|        6|     BVH|Học viện Công ngh...|  Hà Nội|
|        7|     BVS|Học viện Công ngh...|  TP HCM|
|        8|     CCM|Đại học Công Nghi...|  Hà Nội|
|        9|     CEA|Đại học Kinh Tế N...| Nghệ An|
|       10|     CSH|Học viện Cảnh sát...|  Hà Nội|
+---------+--------+--------------------+--------+
only showing top 10 rows



### 2.4. Dim_Major - Bảng ngành học

In [5]:
print("Đang load Dim_Major...")

# Đọc dữ liệu từ Silver
df_major_silver = spark.table("nessie.silver_tables.major")

# Tạo majorKey tự động tăng
window_spec = Window.orderBy("majorId")
df_dim_major = df_major_silver.withColumn("majorKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_major = df_dim_major.select("majorKey", "majorId", "majorName")

# Ghi vào Gold layer
df_dim_major.writeTo("nessie.gold_tables.dim_major") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_major.count()} dòng vào dim_major")
df_dim_major.show(10)

Đang load Dim_Major...
Đã load 3085 dòng vào dim_major
+--------+-------+--------------------+
|majorKey|majorId|           majorName|
+--------+-------+--------------------+
|       1|    106|   Khoa học Máy tính|
|       2|    107|   Kỹ thuật Máy tính|
|       3|    108|Điện - Điện tử - ...|
|       4|    109|     Kỹ Thuật Cơ khí|
|       5|    110| Kỹ Thuật Cơ Điện tử|
|       6|    112|           Dệt - May|
|       7|    114|Hoá - Thực phẩm -...|
|       8|    115|Xây dựng và Quản ...|
|       9|    117|           Kiến Trúc|
|      10|    120|  Dầu khí - Địa chất|
+--------+-------+--------------------+
only showing top 10 rows



### 2.5. Dim_Subject - Bảng môn học

In [6]:
print("Đang load Dim_Subject...")

# Đọc dữ liệu từ Silver
df_subject_silver = spark.table("nessie.silver_tables.subject")

# Tạo subjectKey tự động tăng (sử dụng subjectId làm sort key)
window_spec = Window.orderBy("subjectId")
df_dim_subject = df_subject_silver.withColumn("subjectKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_subject = df_dim_subject.select("subjectKey", "subjectId", "subjectName")

# Ghi vào Gold layer
df_dim_subject.writeTo("nessie.gold_tables.dim_subject") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_subject.count()} dòng vào dim_subject")
df_dim_subject.show(10)

Đang load Dim_Subject...
Đã load 51 dòng vào dim_subject
+----------+---------+--------------------+
|subjectKey|subjectId|         subjectName|
+----------+---------+--------------------+
|         1|        1|biểu diễn nghệ thuật|
|         2|        2|Công nghệ công ng...|
|         3|        3|Công nghệ nông ng...|
|         4|        4|                GDCD|
|         5|        5|                 Hát|
|         6|        6|           Hát & Múa|
|         7|        7|Hát hoặc biểu diễ...|
|         8|        8|            Hình họa|
|         9|        9|                 Hóa|
|        10|       10|     Khoa học xã hội|
+----------+---------+--------------------+
only showing top 10 rows



### 2.6. Dim_Subject_Group - Bảng khối thi

In [7]:
print("Đang load Dim_Subject_Group...")

# Đọc dữ liệu từ Silver
df_subject_group_silver = spark.table("nessie.silver_tables.subject_group")

# Tạo subjectGroupKey tự động tăng
window_spec = Window.orderBy("subjectGroupId")
df_dim_subject_group = df_subject_group_silver.withColumn("subjectGroupKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_subject_group = df_dim_subject_group.select(
    "subjectGroupKey", 
    "subjectGroupId", 
    "subjectGroupName", 
    "subjectCombination"
)

# Ghi vào Gold layer
df_dim_subject_group.writeTo("nessie.gold_tables.dim_subject_group") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_subject_group.count()} dòng vào dim_subject_group")
df_dim_subject_group.show(10)

Đang load Dim_Subject_Group...
Đã load 232 dòng vào dim_subject_group
+---------------+--------------+----------------+--------------------+
|subjectGroupKey|subjectGroupId|subjectGroupName|  subjectCombination|
+---------------+--------------+----------------+--------------------+
|              1|             1|             A00|         Toán-Lí-Hóa|
|              2|             2|             A01|   Toán-Lí-Ngoại ngữ|
|              3|             3|             A02|        Toán-Lí-Sinh|
|              4|             4|             A03|          Toán-Lí-Sử|
|              5|             5|             A04|         Toán-Lí-Địa|
|              6|             6|             A05|         Toán-Hóa-Sử|
|              7|             7|             A06|        Toán-Hóa-Địa|
|              8|             8|             A07|         Toán-Sử-Địa|
|              9|             9|             A0C|Toán-Lí-Công nghệ...|
|             10|            10|             A0T|     Toán-Lí-Tin học|
+------

### 2.7. Dim_Selection_Method - Bảng phương thức xét tuyển

In [8]:
print("Đang load Dim_Selection_Method...")

# Đọc dữ liệu từ Silver
df_selection_method_silver = spark.table("nessie.silver_tables.selection_method")

# Tạo selectionMethodKey tự động tăng
window_spec = Window.orderBy("selectionMethodId")
df_dim_selection_method = df_selection_method_silver.withColumn("selectionMethodKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_selection_method = df_dim_selection_method.select(
    "selectionMethodKey", 
    "selectionMethodId", 
    "selectionMethodName"
)

# Ghi vào Gold layer
df_dim_selection_method.writeTo("nessie.gold_tables.dim_selection_method") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_selection_method.count()} dòng vào dim_selection_method")
df_dim_selection_method.show(10)

Đang load Dim_Selection_Method...
Đã load 10 dòng vào dim_selection_method
+------------------+-----------------+--------------------+
|selectionMethodKey|selectionMethodId| selectionMethodName|
+------------------+-----------------+--------------------+
|                 1|                1|Điểm chuẩn theo p...|
|                 2|                2|Điểm chuẩn theo p...|
|                 3|                3|Điểm chuẩn theo p...|
|                 4|                4|Điểm chuẩn theo p...|
|                 5|                5|Điểm chuẩn theo p...|
|                 6|                6|Điểm chuẩn theo p...|
|                 7|                7|Điểm chuẩn theo p...|
|                 8|                8|Điểm chuẩn theo p...|
|                 9|                9|Điểm chuẩn theo p...|
|                10|               10|Điểm chuẩn theo p...|
+------------------+-----------------+--------------------+



In [9]:
print("Đang load Dim_Grading_Scale...")

# 1. Đọc dữ liệu từ Silver
df_grading_scale_silver = spark.table("nessie.silver_tables.grading_scale")

# 2. Tạo gradingScaleKey tự động tăng (surrogate key)
window_spec = Window.orderBy("gradingScaleId")

df_dim_grading_scale = df_grading_scale_silver.withColumn(
    "gradingScaleKey",
    row_number().over(window_spec)
)

# 3. Sắp xếp lại cột đúng thứ tự trong GOLD schema
df_dim_grading_scale = df_dim_grading_scale.select(
    "gradingScaleKey",
    "gradingScaleId",
    "value",
    "description"
)

# 4. Ghi vào Gold layer
df_dim_grading_scale.writeTo("nessie.gold_tables.dim_grading_scale") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_grading_scale.count()} dòng vào dim_grading_scale")
df_dim_grading_scale.show(10, truncate=False)


Đang load Dim_Grading_Scale...


Đã load 10 dòng vào dim_grading_scale
+---------------+--------------+------+---------------+
|gradingScaleKey|gradingScaleId|value |description    |
+---------------+--------------+------+---------------+
|1              |0             |30.0  |Thang điểm 30  |
|2              |1             |50.0  |Thang điểm 50  |
|3              |2             |1200.0|Thang điểm 1200|
|4              |3             |150.0 |Thang điểm 150 |
|5              |4             |40.0  |Thang điểm 40  |
|6              |5             |10.0  |Thang điểm 10  |
|7              |6             |100.0 |Thang điểm 100 |
|8              |7             |35.0  |Thang điểm 35  |
|9              |8             |90.0  |Thang điểm 90  |
|10             |9             |120.0 |Thang điểm 120 |
+---------------+--------------+------+---------------+



## 3. Load Fact Tables từ Silver Layer

### 3.1. Fact_Benchmark - Load và join với các Dimension tables

In [10]:
print("Đang load Fact_Benchmark...")

# Đọc dữ liệu từ Silver
df_benchmark_silver = spark.table("nessie.silver_tables.benchmark")

# Đọc các dimension tables từ Gold để join
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_school_gold = spark.table("nessie.gold_tables.dim_school")
df_dim_major_gold = spark.table("nessie.gold_tables.dim_major")
df_dim_subject_group_gold = spark.table("nessie.gold_tables.dim_subject_group")
df_dim_selection_method_gold = spark.table("nessie.gold_tables.dim_selection_method")
df_dim_grading_scale_gold = spark.table("nessie.gold_tables.dim_grading_scale")

# Join từng bước để tránh Cartesian product
# Bước 1: Join với dim_time
df_fact = df_benchmark_silver.join(
    df_dim_time_gold, 
    df_benchmark_silver["year"] == df_dim_time_gold["year"], 
    "left"
).drop(df_dim_time_gold["year"])

# Bước 2: Join với dim_school
df_fact = df_fact.join(
    df_dim_school_gold, 
    df_fact["schoolId"] == df_dim_school_gold["schoolId"], 
    "left"
).drop(df_dim_school_gold["schoolId"])

# Bước 3: Join với dim_major
df_fact = df_fact.join(
    df_dim_major_gold, 
    df_fact["majorId"] == df_dim_major_gold["majorId"], 
    "left"
).drop(df_dim_major_gold["majorId"])

# Bước 4: Join với dim_subject_group
df_fact = df_fact.join(
    df_dim_subject_group_gold, 
    df_fact["subjectGroupId"] == df_dim_subject_group_gold["subjectGroupId"], 
    "left"
).drop(df_dim_subject_group_gold["subjectGroupId"])

# Bước 5: Join với dim_selection_method
df_fact = df_fact.join(
    df_dim_selection_method_gold, 
    df_fact["selectionMethodId"] == df_dim_selection_method_gold["selectionMethodId"], 
    "left"
).drop(df_dim_selection_method_gold["selectionMethodId"])

# Bước 6: Join với dim_grading_scale
df_fact = df_fact.join(
    df_dim_grading_scale_gold,
    df_fact["gradingScaleId"] == df_dim_grading_scale_gold["gradingScaleId"],
    "left"
).drop(df_dim_grading_scale_gold["gradingScaleId"])

print(f"Sau tất cả join: {df_fact.count():,} dòng")

# Tạo benchmarkKey tự động tăng
window_spec = Window.orderBy("timeKey", "schoolKey", "majorKey")
df_fact = df_fact.withColumn("benchmarkKey", row_number().over(window_spec))

# Tính điểm trung bình cho mỗi ngành theo từng phương thức
window_avg_spec = Window.partitionBy("timeKey", "schoolKey", "majorKey", "selectionMethodKey", "gradingScaleKey")
df_fact = df_fact.withColumn(
    "avgScoreByMajor",
    round(avg("score").over(window_avg_spec), 2)
)

# Tính toán yearlyScoreGap
window_year_spec = Window.partitionBy("schoolKey", "majorKey", "selectionMethodKey", "gradingScaleKey","subjectGroupKey").orderBy("timeKey")
df_fact = df_fact.withColumn(
    "yearlyScoreGap",
    coalesce(round(col("avgScoreByMajor") - lag("avgScoreByMajor", 1).over(window_year_spec),2), lit(0))
)

# Xếp hạng ngành trong trường
window_major_in_school_spec = Window.partitionBy("timeKey", "schoolKey", "selectionMethodKey", "gradingScaleKey").orderBy(desc("avgScoreByMajor"), "majorKey")
df_fact = df_fact.withColumn("rankAmongMajors", dense_rank().over(window_major_in_school_spec))

# Xếp hạng ngành so với các trường khác
window_major_across_schools_spec = Window.partitionBy("timeKey", "majorKey", "selectionMethodKey", "gradingScaleKey").orderBy(desc("avgScoreByMajor"), "schoolKey")
df_fact = df_fact.withColumn("rankAmongSchools", dense_rank().over(window_major_across_schools_spec))

# Select các cột cuối cùng
df_fact_benchmark = df_fact.select(
    "benchmarkKey",
    "subjectGroupKey",
    "timeKey",
    "majorKey",
    "schoolKey",
    "selectionMethodKey",
    "gradingScaleKey",
    col("score").cast("float"),
    col("avgScoreByMajor").cast("float"),
    col("yearlyScoreGap").cast("float"),
    "rankAmongMajors",
    "rankAmongSchools"
)

# Ghi vào Gold layer
df_fact_benchmark.writeTo("nessie.gold_tables.fact_benchmark") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_benchmark.count()} dòng vào fact_benchmark")
df_fact_benchmark.show(10)

Đang load Fact_Benchmark...


Sau tất cả join: 163,399 dòng


Đã load 163399 dòng vào fact_benchmark


[Stage 102:>                                                        (0 + 1) / 1]

+------------+---------------+-------+--------+---------+------------------+---------------+-----+---------------+--------------+---------------+----------------+
|benchmarkKey|subjectGroupKey|timeKey|majorKey|schoolKey|selectionMethodKey|gradingScaleKey|score|avgScoreByMajor|yearlyScoreGap|rankAmongMajors|rankAmongSchools|
+------------+---------------+-------+--------+---------+------------------+---------------+-----+---------------+--------------+---------------+----------------+
|       14128|              1|      1|    NULL|      179|                 2|              1|23.25|          23.25|           0.0|              9|               1|
|       14129|              2|      1|    NULL|      179|                 2|              1|23.25|          23.25|           0.0|              9|               1|
|       14130|             35|      1|    NULL|      179|                 2|              1|23.25|          23.25|           0.0|              9|               1|
|       14127|        

### 3.2. Fact_Score_Distribution_By_Subject - Phân bố điểm theo môn

In [11]:
print("Đang load Fact_Score_Distribution_By_Subject...")

# Đọc dữ liệu từ Silver
df_student_scores = spark.table("nessie.silver_tables.student_scores")

# Đọc dimension tables
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_region_gold = spark.table("nessie.gold_tables.dim_region")
df_dim_subject_gold = spark.table("nessie.gold_tables.dim_subject")

# Explode map scores thành từng dòng (subjectId - điểm)
# Vì scores giờ là Map<Int, Float> (subjectId -> score)
# subjectId ở đây là INTEGER, chính là cột subjectId từ bảng silver.subject
df_scores_exploded = df_student_scores.select(
    "regionId",
    "year",
    explode("scores").alias("subjectId", "score")  # subjectId = key của map (Int)
)

# Phân loại điểm vào các mốc điểm (0.2, 0.4, 0.6, ..., 9.8, 10.0)
# Làm tròn điểm đến bội số của 0.2 gần nhất
df_scores_exploded = df_scores_exploded.withColumn(
    "scoreThreshold",
    (floor(col("score") / 0.2) * 0.2).cast("float")
)

# Đếm số lượng học sinh theo từng môn, khu vực, năm và ngưỡng điểm
df_distribution = df_scores_exploded.groupBy(
    "regionId",
    "subjectId",  # Đây là subjectId (Int) từ scores map
    "year",
    "scoreThreshold"
).agg(
    count("*").alias("quantity")
)

# Join với các dimension tables
# Join với dim_subject bằng subjectId để lấy subjectKey
df_fact = df_distribution \
    .join(df_dim_region_gold, df_distribution["regionId"] == df_dim_region_gold["regionId"], "left") \
    .join(df_dim_subject_gold, df_distribution["subjectId"] == df_dim_subject_gold["subjectId"], "left") \
    .join(df_dim_time_gold, df_distribution["year"] == df_dim_time_gold["year"], "left")

# Tạo SDBSKey tự động tăng
window_spec = Window.orderBy("timeKey", "regionKey", "subjectKey", "scoreThreshold")
df_fact = df_fact.withColumn("SDBSKey", row_number().over(window_spec))

# Select các cột cuối cùng
# Lưu ý: Lấy subjectKey từ df_dim_subject_gold (không phải subjectId)
df_fact_score_dist_subject = df_fact.select(
    "SDBSKey",
    df_dim_region_gold["regionKey"],
    df_dim_subject_gold["subjectKey"],  # subjectKey từ dim_subject (surrogate key)
    df_dim_time_gold["timeKey"],
    col("scoreThreshold").cast("float"),
    col("quantity").cast("float")
)

# Ghi vào Gold layer
df_fact_score_dist_subject.writeTo("nessie.gold_tables.fact_score_distribution_by_subject") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_score_dist_subject.count()} dòng vào fact_score_distribution_by_subject")
df_fact_score_dist_subject.show(10)

Đang load Fact_Score_Distribution_By_Subject...


Đã load 100187 dòng vào fact_score_distribution_by_subject


[Stage 128:==========================================>              (3 + 1) / 4]

+-------+---------+----------+-------+--------------+--------+
|SDBSKey|regionKey|subjectKey|timeKey|scoreThreshold|quantity|
+-------+---------+----------+-------+--------------+--------+
|      1|     NULL|         9|      1|           0.6|     1.0|
|      2|     NULL|         9|      1|           1.4|     2.0|
|      3|     NULL|         9|      1|           1.6|     4.0|
|      4|     NULL|         9|      1|           2.0|     8.0|
|      5|     NULL|         9|      1|           2.2|     7.0|
|      6|     NULL|         9|      1|           2.4|    22.0|
|      7|     NULL|         9|      1|           2.6|    27.0|
|      8|     NULL|         9|      1|           3.0|    54.0|
|      9|     NULL|         9|      1|           3.2|    60.0|
|     10|     NULL|         9|      1|           3.4|    84.0|
+-------+---------+----------+-------+--------------+--------+
only showing top 10 rows



### 3.3. Fact_Score_Distribution_By_Subject_Group - Phân bố điểm theo khối thi

In [12]:
print("Đang load Fact_Score_Distribution_By_Subject_Group...")

# Đọc dữ liệu từ Silver layer
df_student_scores = spark.table("nessie.silver_tables.student_scores")
df_subject_group = spark.table("nessie.silver_tables.subject_group")
df_subject = spark.table("nessie.silver_tables.subject")

# Đọc dimension tables từ Gold
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_region_gold = spark.table("nessie.gold_tables.dim_region")
df_dim_subject_group_gold = spark.table("nessie.gold_tables.dim_subject_group")

# Tạo mapping dictionaries
subject_name_to_id = {row['subjectName']: row['subjectId'] for row in df_subject.collect()}
subject_groups_dict = {}
for row in df_subject_group.collect():
    group_id = row['subjectGroupId']
    subjects = [s.strip() for s in row['subjectCombination'].split('-')]
    subject_groups_dict[group_id] = subjects

# Broadcast để worker nodes truy cập nhanh
subject_map_broadcast = spark.sparkContext.broadcast(subject_name_to_id)
subject_groups_broadcast = spark.sparkContext.broadcast(subject_groups_dict)

# UDF tính điểm tổng theo khối thi
def create_subject_group_scores_map(scores_map):
    if scores_map is None:
        return {}
    
    subject_name_to_id_map = subject_map_broadcast.value
    subject_groups = subject_groups_broadcast.value
    result = {}
    
    for group_id, required_subjects in subject_groups.items():
        total_score = 0.0
        has_all_subjects = True
        
        for subject_name in required_subjects:
            subject_id = subject_name_to_id_map.get(subject_name)
            if subject_id is None or subject_id not in scores_map:
                has_all_subjects = False
                break
            total_score += scores_map[subject_id]
        
        if has_all_subjects:
            result[group_id] = total_score
    
    return result

create_group_scores_udf = udf(create_subject_group_scores_map, MapType(StringType(), FloatType()))

# Tạo cột mới chứa mapping subjectGroupId -> totalScore
df_student_with_group_scores = df_student_scores.withColumn(
    "subject_group_scores",
    create_group_scores_udf(col("scores"))
)

# Explode map thành rows
df_scores_exploded = df_student_with_group_scores.select(
    "regionId",
    "year",
    explode("subject_group_scores").alias("subjectGroupId", "totalScore")
)

# Phân loại điểm vào khoảng
df_scores_exploded = df_scores_exploded.withColumn(
    "scoreRange",
    floor(col("totalScore")).cast("float")
)

# Đếm số lượng học sinh
df_distribution = df_scores_exploded.groupBy(
    "regionId",
    "subjectGroupId",
    "year",
    "scoreRange"
).agg(
    count("*").alias("quantity")
)

# Tạo mapping dictionaries từ dimension tables
region_id_to_key = {row['regionId']: row['regionKey'] for row in df_dim_region_gold.select("regionId", "regionKey").collect()}
subject_group_id_to_key = {str(row['subjectGroupId']).strip().upper(): row['subjectGroupKey'] for row in df_dim_subject_group_gold.select("subjectGroupId", "subjectGroupKey").collect()}
year_to_time_key = {row['year']: row['timeKey'] for row in df_dim_time_gold.select("year", "timeKey").collect()}

# Broadcast mappings
region_map_bc = spark.sparkContext.broadcast(region_id_to_key)
subject_group_map_bc = spark.sparkContext.broadcast(subject_group_id_to_key)
year_map_bc = spark.sparkContext.broadcast(year_to_time_key)

# UDF để map các natural keys sang surrogate keys
def map_to_keys(regionId, subjectGroupId, year):
    region_key = region_map_bc.value.get(regionId)
    normalized_group_id = str(subjectGroupId).strip().upper() if subjectGroupId is not None else None
    subject_group_key = subject_group_map_bc.value.get(normalized_group_id)
    time_key = year_map_bc.value.get(year)
    return (region_key, subject_group_key, time_key)

map_keys_udf = udf(map_to_keys, StructType([
    StructField("regionKey", IntegerType(), True),
    StructField("subjectGroupKey", IntegerType(), True),
    StructField("timeKey", IntegerType(), True)
]))

# Áp dụng mapping
df_with_keys = df_distribution.withColumn(
    "keys",
    map_keys_udf(col("regionId"), col("subjectGroupId"), col("year"))
)

# Extract keys từ struct
df_with_keys = df_with_keys.withColumn("regionKey", col("keys.regionKey")) \
    .withColumn("subjectGroupKey", col("keys.subjectGroupKey")) \
    .withColumn("timeKey", col("keys.timeKey")) \
    .drop("keys", "regionId", "subjectGroupId", "year")

# Tạo surrogate key
window_spec = Window.orderBy("timeKey", "regionKey", "subjectGroupKey", "scoreRange")
df_with_keys = df_with_keys.withColumn("SDBSGKey", row_number().over(window_spec))

# Select các cột cuối cùng
df_fact_score_dist_group = df_with_keys.select(
    "SDBSGKey",
    "regionKey",
    "subjectGroupKey",
    "timeKey",
    col("scoreRange").cast("float"),
    col("quantity").cast("float")
)

# Ghi vào Gold layer
df_fact_score_dist_group.writeTo("nessie.gold_tables.fact_score_distribution_by_subject_group") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_score_dist_group.count()} dòng vào fact_score_distribution_by_subject_group")
df_fact_score_dist_group.show(10)

Đang load Fact_Score_Distribution_By_Subject_Group...


Đã load 217784 dòng vào fact_score_distribution_by_subject_group


[Stage 155:===================>                                     (2 + 4) / 6]

+--------+---------+---------------+-------+----------+--------+
|SDBSGKey|regionKey|subjectGroupKey|timeKey|scoreRange|quantity|
+--------+---------+---------------+-------+----------+--------+
|       1|        1|              1|      1|       4.0|     1.0|
|       2|        1|              1|      1|       5.0|     2.0|
|       3|        1|              1|      1|       6.0|    11.0|
|       4|        1|              1|      1|       7.0|    22.0|
|       5|        1|              1|      1|       8.0|    32.0|
|       6|        1|              1|      1|       9.0|    32.0|
|       7|        1|              1|      1|      10.0|    81.0|
|       8|        1|              1|      1|      11.0|   133.0|
|       9|        1|              1|      1|      12.0|   206.0|
|      10|        1|              1|      1|      13.0|   364.0|
+--------+---------+---------------+-------+----------+--------+
only showing top 10 rows



## 4. Dừng Spark Session

In [13]:
# Dừng Spark Session để giải phóng resources
spark.stop()
print(" Spark Session đã được dừng!")

 Spark Session đã được dừng!
